In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle

In [2]:
hgg = pd.read_parquet('/scratch365/cmoore24/training/hgg/batch/inputs/grm_ungrm/flat400_all_ratios.parquet')
qcd = pd.read_parquet('/scratch365/cmoore24/training/hgg/batch/inputs/grm_ungrm/sculpted_qcd_all_ratios.parquet')

In [3]:
hgg['isSignal'] = 1
qcd['isSignal'] = 0

In [4]:
qcd = pd.concat([hgg, qcd])

In [5]:
del(hgg)

In [6]:
feature_names = qcd.columns
feature_names = feature_names[:-1]

In [7]:
X = qcd.drop(columns=['isSignal']).values

In [8]:
y = qcd['isSignal'].values

In [9]:
del(qcd)

In [10]:
rf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rf.fit(X, y)

RandomForestClassifier(n_estimators=500, n_jobs=-1)

In [22]:
importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]
top_n = 100
X_reduced_rf = X[:, indices[:top_n]]

In [23]:
rf_selected_features = feature_names[indices[:top_n]].tolist()

In [24]:
grm = 0
ungrm = 0
for i in rf_selected_features:
    if 'Grm' in i:
        grm += 1
    elif 'Ungrm' in i:
        ungrm += 1

In [25]:
grm, ungrm

(31, 69)

In [14]:
with open('./inputs/grm_ungrm/rf_model.pkl', 'wb') as f:
    pickle.dump(rf, f)